In [120]:
import pandas as pd
import numpy as np
from statsmodels.tsa.filters.hp_filter import hpfilter
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model._ridge import Ridge
from sklearn.metrics import r2_score
from keras.metrics import mean_absolute_percentage_error, RootMeanSquaredError
from sklearn.model_selection import GridSearchCV, train_test_split
import warnings
from xgboost import XGBRegressor
warnings.filterwarnings('ignore')


In this notebook, I will explore the difference between applying regression model (Kernel ridge regression) using different sets of data : a raw data (closing price of stocks daily), and a preprocessed smoothed data (closing price of stocks w/ HP-Filter applied). For the sake of comparison, I will be using two different assets, which are TESLA and US Treasury bond. 

In [59]:
days = 250 # This parameter sets different number of days we are going to use for our time lag

We apply timelag to our data in order to generate more data from a single source. This technique is commonly used in time series analysis.

In [60]:

def produceHP(ticker):
    ticker = "\\Data\\{}".format(ticker) + ".csv"
    dir = os.getcwd() + ticker
    data = pd.read_csv(dir)
    data = data.set_index('Date')
    data['preprice'] = data['Close'].T.shift(1)
    data = data.dropna()
    data['Close'] = (data['Close'] - data['preprice']) / data['Close']
    cycle, trend = sm.tsa.filters.hpfilter(data['Close'], 10000000)
    x = data.index
    x = [dt.datetime.strptime(d, '%Y-%m-%d') for d in x]
    y = trend
    # plt.plot(x, y)
    # y = data['Close']
    # plt.plot(x, y)
    # plt.show()
    return (data, trend, cycle)

produceHP("TSLA")
def reshape_data(df):
    data_reshape = pd.concat([df['Close'].T.shift(i).to_frame().stack(dropna=False) for i in range(days) ], 1).dropna()
    data_reshape.columns = pd.Index(range(days), name='timeLag')
    return data_reshape

First, we apply HP-filter to a data and see what it does in order to understand it. Then, we apply time lag to the raw data.

In [61]:
data, trend, cycle = produceHP("TSLA")
data_reshape = reshape_data(data)

We define a split method in order to set some of the data as test set, and some as training set.

In [125]:
def split(data):
    X = data[np.arange(1, days)]
    y = data[0]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.25)
    return (X_train, X_test, X_valid, y_train, y_test, y_valid)

Then, we apply time lag onto HP-Filter too.

In [63]:
def get_HP_lag(data_reshape):
    trend_data = pd.DataFrame()
    # trend_data.columns = pd.Index(range(50), name='timeLag')
    rows = []
    for i in range(len(data_reshape)):
        row = data_reshape.iloc[i]
        cycle, trend = sm.tsa.filters.hpfilter(row[1:], 100000)
        trend = pd.concat([cycle, trend], axis = 0)
        trend.index = np.arange(1, len(cycle) * 2 + 1)
        rows.append(pd.concat([pd.Series(row[0]), trend], axis = 0))
    
    trend_data = pd.DataFrame(rows)
    trend_data.columns = pd.Index(range(days * 2 - 1), name='timeLag')
    return trend_data

Since we now have all data that we need for comparison, we run kernel-ridge regression to test how the model performs. We decided to use this model since it supports non-linearity of the data. Hyperparameter, also known as a regularization factor can be adjusted accordingly. If regularization term is 0, it becomes a traditional least-square problem, while higher regularization factor underfits the model. Thus, there are two hyperparameters in our model : number of days used for prediction, and regularization factor alpha.

In [129]:
def test(data_reshape, reg):
    sum = 0.0
    for i in range(10):
        X_train, X_test, X_valid, y_train, y_test, y_valid = split(data_reshape)
        model = XGBRegressor()
        model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds=5)
        prediction = model.predict(X_test)
        m = RootMeanSquaredError()
        m.update_state(y_test, prediction)
        sum += m.result()
    return sum.numpy()

In [65]:
ticker = "TSLA"
days = 250

In [127]:
def pipeline(t, d, a):
    ticker = t
    days = d
    data, trend, cycle = produceHP(ticker)
    data_reshape = reshape_data(data)
    trend_reshape = get_HP_lag(data_reshape)
    print(trend_reshape)
    return (test(data_reshape, a), test(trend_reshape, a))

In [130]:
pipeline("TSLA", 250, 1)

timeLag       0         1         2         3         4         5         6    \
0       -0.003215  0.055703 -0.029111  0.096367 -0.080489  0.015597 -0.054085   
1       -0.073133 -0.000436  0.055738 -0.029078  0.096398 -0.080459  0.015626   
2       -0.032494 -0.064813  0.004716  0.060678 -0.024349  0.100917 -0.076147   
3        0.054550 -0.021968 -0.063067  0.006390  0.062280 -0.022818  0.102379   
4        0.051558  0.060527 -0.026779 -0.067680  0.001974  0.058060 -0.026844   
..            ...       ...       ...       ...       ...       ...       ...   
941      0.002464 -0.042974 -0.037456 -0.021844  0.003086  0.022364  0.002211   
942      0.024493  0.007374 -0.043560 -0.038018 -0.022382  0.002572  0.021873   
943      0.016921  0.027441  0.005193 -0.045651 -0.040020 -0.024295  0.000746   
944     -0.073077  0.018553  0.025966  0.003779 -0.047005 -0.041314 -0.025529   
945     -0.011159 -0.065836  0.023787  0.030984  0.008582 -0.042414 -0.036934   

timeLag       7         8  

AssertionError: Must have at least 1 validation dataset for early stopping.

As you can see, RMSE for model using raw data was higher compared to its counterpart using smoothed data. 

In [110]:
pipeline("^TNX", 250, 1)

timeLag       0         1         2         3         4         5         6    \
0       -0.018615  0.003414 -0.013842  0.022877 -0.010055  0.006533  0.009274   
1       -0.009395 -0.011557  0.004333 -0.012962  0.023720 -0.009249  0.007302   
2       -0.041895 -0.002005 -0.011398  0.004486 -0.012815  0.023860 -0.009116   
3        0.039489 -0.031707  0.000516 -0.008981  0.006799 -0.010605  0.025969   
4        0.008576  0.046135 -0.035374 -0.003000 -0.012347  0.003582 -0.013674   
..            ...       ...       ...       ...       ...       ...       ...   
941      0.046674 -0.014358  0.042317 -0.028158  0.012205  0.001859 -0.013064   
942      0.021695  0.032314 -0.016927  0.039854 -0.030516  0.009953 -0.000291   
943     -0.069906  0.006397  0.031805 -0.017415  0.039387 -0.030962  0.009527   
944      0.011209 -0.079085  0.012685  0.037833 -0.011645  0.044901 -0.025700   
945      0.014984  0.001718 -0.079222  0.012554  0.037707 -0.011765  0.044787   

timeLag       7         8  

(0.49230874, 0.45733306)

In [111]:
d = np.arange(10, 251, 40)
a = [0.1, 0.01, 1, 10]
rawMin = (0, 0)
trendMin = (0, 0)
rawScore = 999999
trendScore = 999999
for day in d:
    days = day
    ticker = "TSLA"
    data, trend, cycle = produceHP(ticker)
    data_reshape = reshape_data(data)
    trend_reshape = get_HP_lag(data_reshape)
    for alpha in a:
        raw = test(data_reshape, alpha)
        raw = raw
        trend = test(trend_reshape, alpha)
        trend = trend
        if raw < rawScore:
            rawScore = raw
            rawMin = (day, alpha)
        if trend < trendScore:
            trendScore = trend
            trendMin = (day, alpha)
print(rawMin)
print(trendMin)

(10, 10)
(50, 10)


In [112]:
print(rawScore)
print(trendScore)

0.39556122
0.40308732


In [113]:
rawMin = (0, 0)
trendMin = (0, 0)
rawScore = 999999
trendScore = 999999
for day in d:
    days = day
    ticker = "NVDA"
    data, trend, cycle = produceHP(ticker)
    data_reshape = reshape_data(data)
    trend_reshape = get_HP_lag(data_reshape)
    for alpha in a:
        raw = test(data_reshape, alpha)
        raw = raw
        trend = test(trend_reshape, alpha)
        trend = trend
        if raw < rawScore:
            rawScore = raw
            rawMin = (day, alpha)
        if trend < trendScore:
            trendScore = trend
            trendMin = (day, alpha)
print(rawMin)
print(trendMin)

(10, 10)
(10, 0.1)


In [114]:
print(rawScore)
print(trendScore)

0.32349828
0.32211268


In [115]:
rawMin = (0, 0)
trendMin = (0, 0)
rawScore = 999999
trendScore = 999999
for day in d:
    days = day
    ticker = "^TNX"
    data, trend, cycle = produceHP(ticker)
    data_reshape = reshape_data(data)
    trend_reshape = get_HP_lag(data_reshape)
    for alpha in a:
        raw = test(data_reshape, alpha)
        raw = raw
        trend = test(trend_reshape, alpha)
        trend = trend
        if raw < rawScore:
            rawScore = raw
            rawMin = (day, alpha)
        if trend < trendScore:
            trendScore = trend
            trendMin = (day, alpha)
print(rawMin)
print(trendMin)

(10, 0.1)
(10, 1)


In [116]:
print(rawScore)
print(trendScore)

0.37151995
0.3774387


In [117]:
rawMin = (0, 0)
trendMin = (0, 0)
rawScore = 999999
trendScore = 999999
for day in d:
    days = day
    ticker = "TWTR"
    data, trend, cycle = produceHP(ticker)
    data_reshape = reshape_data(data)
    trend_reshape = get_HP_lag(data_reshape)
    for alpha in a:
        raw = test(data_reshape, alpha)
        raw = raw
        trend = test(trend_reshape, alpha)
        trend = trend
        if raw < rawScore:
            rawScore = raw
            rawMin = (day, alpha)
        if trend < trendScore:
            trendScore = trend
            trendMin = (day, alpha)
print(rawMin)
print(trendMin)
print(rawScore)
print(trendScore)

(210, 1)
(210, 10)
0.324441
0.3123093


In [118]:
rawMin = (0, 0)
trendMin = (0, 0)
rawScore = 999999
trendScore = 999999
for day in d:
    days = day
    ticker = "AXON"
    data, trend, cycle = produceHP(ticker)
    data_reshape = reshape_data(data)
    trend_reshape = get_HP_lag(data_reshape)
    for alpha in a:
        raw = test(data_reshape, alpha)
        raw = raw
        trend = test(trend_reshape, alpha)
        trend = trend
        if raw < rawScore:
            rawScore = raw
            rawMin = (day, alpha)
        if trend < trendScore:
            trendScore = trend
            trendMin = (day, alpha)
print(rawMin)
print(trendMin)
print(rawScore) 
print(trendScore)

(10, 0.1)
(10, 0.01)
0.31740874
0.32639754


In [119]:
rawMin = (0, 0)
trendMin = (0, 0)
rawScore = 999999
trendScore = 999999
for day in d:
    days = day
    ticker = "FDP"
    data, trend, cycle = produceHP(ticker)
    data_reshape = reshape_data(data)
    trend_reshape = get_HP_lag(data_reshape)
    for alpha in a:
        raw = test(data_reshape, alpha)
        raw = raw
        trend = test(trend_reshape, alpha)
        trend = trend
        if raw < rawScore:
            rawScore = raw
            rawMin = (day, alpha)
        if trend < trendScore:
            trendScore = trend
            trendMin = (day, alpha)
print(rawMin)
print(trendMin)
print(rawScore) 
print(trendScore)

(50, 10)
(10, 0.1)
0.25987822
0.2543211
